# Why use Azure machine learning(ML) pipelines to run your flows on the cloud?
In real-world scenarios, flows serve various purposes. For example, consider a flow designed to evaluate the relevance score for a communication session between humans and agents. Suppose you want to trigger this flow every night to assess today’s performance and avoid peak hours for LLM (Language Model) endpoints. In this common scenario, people often encounter the following needs:
- Handling Large Data Inputs: Running flows with thousands or millions of data inputs at once.
- Scalability and Efficiency: Requiring a scalable, efficient, and resilient platform to ensure success.
- Automations: Automatically triggering batch flows when upstream data is ready or at fixed intervals.

__Azure ML pipelines__ address all these offline requirements effectively. With the integration of prompt flows and Azure ML pipeline, flow users could very easily achieve above goals and in this tutorial, you can learn:
- How to use python SDK to automatically convert your flow into a 'step' in Azure ML pipeline.
- How to feed your data into pipeline to trigger the batch flow runs.
- How to build other pipeline steps ahead or behind your prompt flow step. e.g. data preprocessing or result aggregation.
- How to setup a simple scheduler on my pipeline.
- How to deploy pipeline to an Azure ML batch endpoint. Then I can invoke it with new data when needed.

Before you begin, consider the following prerequisites:
- Introduction to Azure ML Platform:
    - [Core site of Azure ML platform](https://learn.microsoft.com/en-us/azure/machine-learning/overview-what-is-azure-machine-learning?view=azureml-api-2).
    - Understand what [Azure ML pipelines](https://learn.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines?view=azureml-api-2) and [component](https://learn.microsoft.com/en-us/azure/machine-learning/concept-component?view=azureml-api-2) are.
- Azure cloud setup:
    - An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
    - Create an Azure ML resource from Azure portal - [Create a Azure ML workspace](https://ms.portal.azure.com/#view/Microsoft_Azure_Marketplace/MarketplaceOffersBlade/searchQuery/machine%20learning)
    - Connect to your workspace then setup a basic computer cluster - [Configure workspace](../../configuration.ipynb)
- Local environment setup:
    - A python environment
    - Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, load_component, Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use 'config file' to connect to your workspace. Check [this notebook](../../configuration.ipynb) to get your config file from Azure ML workspace portal and paste it into this folder. Then if you pass the next code block, you've all set for the environment.

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Load flow as component
If you’ve already authored a flow using the Promptflow SDK or portal, you can locate the flow.dag.yaml file within the flow folder. This YAML specification is essential for loading your flow into an Azure ML component.

In [ ]:
flow_component = load_component("../../flows/standard/web-classification/flow.dag.yaml")

When using the `load_component` function and the flow YAML specification, your flow is automatically transformed into a __[parallel component](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-parallel-job-in-pipeline?view=azureml-api-2&tabs=cliv2#why-are-parallel-jobs-needed)__. This parallel component is designed for large-scale, offline, parallelized processing with efficiency and resilience. Here are some key features of this auto-converted component:

 - Pre-defined input and output ports:

    ![prompt flow base component image](../../../docs/media/cloud/flow-in-pipeline/pf-base-component.png)
    | port name  |  type  | description |
    | ---------- | ------ | ----------- |
    | data | uri_folder or uri_file | Accepts batch data input to your flow. You can use either the `uri_file` data type if your data is a single file or the `uri_folder` data type if your folder contains multiple files with the same schema. The default data type is jsonl, but you can customize this setting after declaring an instance of this flow component in your pipeline. Note that your data will be converted into a dataframe, so ensure that your CSV or TSV data includes a header line for proper mapping. |
    | flow_outputs | uri_file | Generates a single output file named parallel_run_step.jsonl. Each line in this data file corresponds to a JSON object representing the flow returns, along with an additional column called line_number indicating its position from the original file. |
    | debug_info | uri_folder | If you run your flow component in __debug mode__, this port provides debugging information for each run of your lines. E.g. intermediate outputs between steps, or LLM response and token usage. |

 - Auto-generated parameters 
 
   These parameters represent all your flow inputs and connections associated with your flow steps. You can set default values in the flow/run definition, and they can be further customized during job submission. Use '[web-classification](../../flows/standard/web-classification/flow.dag.yaml)' sample flow for example, this flow has only one input named 'url' and 2 LLM steps 'summarize_text_content' and 'classify_with_llm'. The input parameters of this flow component are:
 
   ![prompt flow base component image](../../../docs/media/cloud/flow-in-pipeline/pf-component-parameters.png)

# 3. Build your pipeline
## 3.1 Declare input and output
To supply your pipeline with data, you need to declare an input using the `path`, `type`, and `mode` properties. Please note: `mount` is the default and suggested mode for your file or folder data input.

Declaring the pipeline output is optional. However, if you require a customized output path in the cloud, you can follow the example below to set the path on the datastore. For more detailed information on valid path values, refer to this documentation - [manage pipeline inputs outputs](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-inputs-outputs-pipeline?view=azureml-api-2&tabs=cli#path-and-mode-for-data-inputsoutputs).

In [ ]:
data_input = Input(
    path="../../flows/standard/web-classification/data.jsonl", 
    type=AssetTypes.URI_FILE,
    mode="mount"
)

pipeline_output = Output(
    # Provide custom flow output file path if needed
    # path="azureml://datastores/<data_store_name>/paths/<path>",
    type=AssetTypes.URI_FOLDER,
    # rw_mount is suggested for flow output
    mode="rw_mount",
)

# 3.2.1 Run pipeline with single flow component
Since all Promptflow components are based on Azure ML parallel components, users can leverage specific __run settings__  to control the parallelization of flow runs. Below are some useful settings:

| run settings | description | allowed values | default value |
| ------------ | ----------- | -------------- | ------------- |
| PF_INPUT_FORMAT | When utilizing `uri_folder` as the input data, this setting allows you to specify which file extensions should be treated as data files for initializing flow runs. | json, jsonl, csv, tsv | jsonl |
| compute | Defines which compute cluster from your Azure ML workspace will be used for this job. | | |
| instance_count | Define how many nodes from your compute cluster will be assigned to this job. | from 1 to node count of compute cluster. | 1 |
| max_concurrency_per_instance | Defines how many dedicated processors will run the flow in parallel on 1 node. When combined with the 'instance_count' setting, the total parallelization of your flow will be instance_count*max_concurrency_per_instance.| >1 | 1 |
| mini_batch_size | Define the number of lines for each mini-batches. A __mini-batch__ is the basic granularity for processing full data with parallelization. Each worker processor handles one mini-batch at a time, and all workers work in parallel across different nodes. | > 0 | 1 |
| max_retries | Defines the retry count if any mini-batch encounters an inner exception. </br></br> Remark: The retry granularity is based on mini-batches. For instance, with the previous setting, you can set 100 lines per mini-batch. When one line execution encounters a transient issue or an unhandled exception, these 100 lines will be retried together, even if the remaining 99 lines are successful. Additionally, LLM responses with status code 429 will be handled internally for flow runs in most cases and will not trigger mini-batch failure. | >= 0 | 3 |
| error_threshold | Defines how many failed lines are acceptable. If the count of failed lines exceeds this threshold, the job will be stopped and marked as failed. Set '-1' to disable this failure check. | -1 or >=0 | -1 |
| mini_batch_error_threshold | Defines the maximum number of failed mini-batches that can be tolerated after all retries. Set '-1' to disable this failure check. | -1 or >=0 | -1 |
| logging_level | Determines how parallel jobs save logs to disk. Setting to ‘DEBUG’ for the flow component allows the component to output intermediate flow logs into the ‘debug_info’ port. | INFO, WARNING, DEBUG | INFO |
| timeout | Sets the timeout checker for each mini-batch execution in milliseconds. If a mini-batch runs longer than this threshold, it will be marked as failed and trigger the next retry. Consider setting a higher value based on your mini-batch size and total traffic throughput for your LLM endpoints. | > 0 | 600 |


In [ ]:
# Define the pipeline as a function
@pipeline()
def pipeline_func_with_flow(
    # Function inputs will be treated as pipeline input data or parameters. 
    # Pipeline input could be linked to step inputs to pass data between steps.
    # Users are not required to define pipeline inputs. 
    # With pipeline inputs, user can provide the different data or values when they trigge different pipeline runs.
    pipeline_input_data : Input,
    parallel_node_count : int = 2,
):
    # Declare pipeline step 'flow_node' by using flow component
    flow_node = flow_component(
        # Bind the pipeline intput data to the port 'data' of the flow component
        # If you don't have pipeline input, you can directly pass the 'data_input' object to the 'data'
        # But with this approach, you can't provide different data when you trigger different pipeline runs.
        # data=data_input,
        data=pipeline_input_data,
        # Declare which column of input data should be mapped to flow input
        # the value pattern follows ${data.<column_name_from_data_input>}
        url="${data.url}",
        # Provide the connection values of the flow component
        # The value of connection and deployment_name should align with your workspace connection settings.
        connections={
            "summarize_text_content": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
            "classify_with_llm": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
        },
    )

    # Provide run settings of your flow component
    # Only 'compute' is required and other setting will keep default value if not provided.
    flow_node.environment_variables = {
        "PF_INPUT_FORMAT": "jsonl",
    }
    flow_node.compute = "cpu-cluster"
    flow_node.resources = { 'instance_count': parallel_node_count }
    flow_node.max_concurrency_per_instance = 2
    flow_node.mini_batch_size = 10
    flow_node.retry_settings = {
       "max_retries": -1,
       "timeout":1200,
    }
    flow_node.error_threshold = -1
    flow_node.mini_batch_error_threshold = -1
    flow_node.logging_level = "DEBUG"

    # Function return will be treated as pipeline output. This is not required.
    return {
        "flow_result_folder" : flow_node.outputs.flow_outputs
    }


# create pipeline instance
pipeline_job = pipeline_func_with_flow(pipeline_input_data=data_input)
pipeline_job.outputs.flow_result_folder = pipeline_output

Submit the pipeline job to your workspace then check the status of your job on UI through the link in the output.

In [ ]:
# Submit the pipeline job to your workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="Single_flow_component_pipeline_job"
)
pipeline_job

# 3.2.2 Run complex pipeline with multiple component
In a typical pipeline, you’ll find multiple steps that encompass all your offline business requirements. If you’re aiming to construct a more intricate pipeline for production, explore the following resources:
 - [how to create component with SDK v2](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python?view=azureml-api-2)
 - Various component types:
    - [Command](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-component-command?view=azureml-api-2)
    - [Spark](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-component-spark?view=azureml-api-2)
    - [Pipeline](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-component-pipeline?view=azureml-api-2)


Additionally, consider the following sample code that loads two extra command components from a repository to construct a single offline pipeline:
 - __data_prep_component__ : This dummy data preprocessing step performs simple data sampling.
 - __result_parser_component__: Combining source data, flow results, and debugging output, it generates a single file containing origin queries, LLM predictions, and LLM token usages.

In [78]:
data_input = Input(
    path="../../flows/standard/web-classification/data.jsonl",
    type=AssetTypes.URI_FILE,
    mode="mount"
)

# load components
data_prep_component = load_component("./components/data-prep/data-prep.yaml")
result_parser_component = load_component("./components/result-parser/result-parser.yaml")

# load flow as component
flow_component = load_component("../../flows/standard/web-classification/flow.dag.yaml")

@pipeline()
def pipeline_func_with_flow(data):
    data_prep_node = data_prep_component(
        input_data_file=data,
    )
    data_prep_node.compute = "cpu-cluster"

    flow_node = flow_component(
        # Feed the output of data_prep_node to the flow component
        data=data_prep_node.outputs.output_data_folder,
        url="${data.url}",
        connections={
            "summarize_text_content": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
            "classify_with_llm": {
                "connection": "azure_open_ai_connection",
                "deployment_name": "gpt-35-turbo",
            },
        },
    )

    flow_node.environment_variables = {"PF_INPUT_FORMAT": "csv"}
    flow_node.compute = "cpu-cluster"
    flow_node.resources = { 'instance_count': 2 }
    flow_node.outputs.flow_outputs.mode = "rw_mount"
    flow_node.logging_level = "DEBUG"

    result_parser_node = result_parser_component(
        source_data=data_prep_node.outputs.output_data_folder,
        pf_output_data=flow_node.outputs.flow_outputs,
        pf_debug_data=flow_node.outputs.debug_info,
    )
    
    result_parser_node.compute = "cpu-cluster"

# create pipeline instance
pipeline_job = pipeline_func_with_flow(data=data_input)

Submit the pipeline job to your workspace then check the status of your job on UI through the link in the output.

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="Complex_flow_component_pipeline_job"
)
pipeline_job

#4. Setup scheduler for your pipeline
TODO

#5. Deploy pipeline to an endpoint
TODO

# Next Steps
You can see further examples of running a pipeline job [here](../)

In [66]:
#### test section 
#### will removed later
import pandas as pd
import glob

input_data = pd.read_csv("./processed_data.csv")
output_data = pd.read_json("./parallel_run_step.jsonl", lines=True)
pf_debug_files = glob.glob("./debug_info/flow_artifacts/*.jsonl")

debug_df = pd.concat([pd.read_json(file, lines=True) for file in pf_debug_files])

output_data.sort_values(by="line_number", inplace=True, ignore_index=True)
debug_df.sort_values(by="line_number", inplace=True, ignore_index=True)

# print(debug_df.head())
# print( [i for i in debug_df.loc[:, "run_info"]] )


input_data.loc[:, "line_number"] = output_data.loc [:, "line_number"]
input_data.loc[:, "pred_category"] = output_data.loc[:, "category"]
input_data.loc[:, "pred_evidence"] = output_data.loc[:, "evidence"]


for i in range(len(debug_df)):
    input_data.loc[i, "prompt_tokens"] = debug_df.loc[i, "run_info"]["system_metrics"]["prompt_tokens"]
    input_data.loc[i, "duration"] = debug_df.loc[i, "run_info"]["system_metrics"]["duration"]
    input_data.loc[i, "completion_tokens"] = debug_df.loc[i, "run_info"]["system_metrics"]["completion_tokens"]
    input_data.loc[i, "total_tokens"] = debug_df.loc[i, "run_info"]["system_metrics"]["total_tokens"]

print(input_data.head())




                                           url    answer evidence  \
0             https://arxiv.org/abs/2307.04767  Academic     Both   
1  https://www.youtube.com/watch?v=kYqRtjDBci8   Channel     Both   
2  https://www.youtube.com/watch?v=kYqRtjDBci8   Channel     Both   
3             https://arxiv.org/abs/2307.04767  Academic     Both   
4             https://arxiv.org/abs/2307.04767  Academic     Both   

   line_number pred_category pred_evidence prompt_tokens  duration  \
0            0      Academic          Both          1162  2.081417   
1            1       Channel          Both           687  1.141858   
2            2          None          None           684  2.037589   
3            3      Academic          Both          1153  2.024096   
4            4      Academic          Both          1162  2.641773   

   completion_tokens  total_tokens  
0              143.0        1305.0  
1               40.0         727.0  
2               37.0         721.0  
3              1

In [ ]:
import glob
import numpy as np

pf_debug_files = glob.glob("./debug_info/flow_artifacts/*.jsonl")

debug_df = pd.concat([pd.read_json(file, lines=True) for file in pf_debug_files])

# print(debug_df)

input_data.loc[:, ["prompt_tokens", "completion_tokens", "total_tokens", "duration"]] = np.nan

print(input_data)


# with open("./processed_data_with_predictions.jsonl", "w") as file:
#     file.write(final_df.to_json(orient="records", lines=True))